In [1]:
EXPERIMENT_IDENTIFIER = "prompt_experiment_1"

In [2]:
N = 5

In [3]:
import pandas as pd
ethnic_df = pd.read_csv("../data/ethnic_dataset_4893_after_removing_rakhain.csv")
ethnic_df

,Unnamed: 0,content
0,0,রমেল হত্যায় মানবাধিকার কমিশনের তদন্ত শুরুরাঙ্গ...
1,1,"নারায়ণগঞ্জের রূপগঞ্জ ও ফতুল্লা, গাজীপুরের কালি..."
2,2,গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগ...
3,3,চাঞ্চল্যকর পিলখানা হত্যা মামলায় ডেথ রেফারেন্স ...
4,4,অবাধে মাটি কেটে নেয়ার ধ্বংসের দ্বারপ্রান্তে ধো...
...,...,...
4888,4888,খাগড়াছড়ির দীঘিনালায় জেএসএস (এমএন লারমা) সমর্থি...
4889,4889,শান্তি-সম্প্রীতি স্থাপনে দুই দশক আগে পার্বত্য ...
4890,4890,জনপ্রিয় ম্যাগাজিন অনুষ্ঠান ‘ইত্যাদি’ আজ শুক্রব...
4891,4891,সরকারি বেতন স্কেলের নবম থেকে ১৩তম গ্রেড পর্যন্...


In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### Loading model

In [5]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
!openai migrate

Your working tree currently has untracked changes and Grit will rewrite files in place. Do you want to proceed? [y/N] 

In [7]:
pip install rich

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [30]:
from openai import OpenAI
import json

client = OpenAI()

def get_response(news_article):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": news_article}
        ]
    )
    return completion.choices[0].message.content
    
def post_process_chatgpt_turbo_3_5_response_to_json(text):
    json_string = text.replace("```", "").replace("json","").replace("\n","").replace("'", '"').replace("\t","")
    #print(f"Midway: {json_string}")
    json_output = json.loads(json_string)
    return json_output

In [53]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()
print(prompt)

You are a Bangla news article analyzer. I will give you a news article and you will say which genre(s) it belongs to, whether it is an article about ethnic people or not, its description style, sentiment and how ethnic people are featured in that article if its an ethnic article.

  The genres are:  Politics, Crime, International Affairs,  Business & Economy,  Sports,  Culture & Entertainment,  Jobs,  Lifestyle,  Health,  Environment,  Local news,  Education,  Technology,  Religion,  Science,  Protests and Social Movements, Govt Actions.
  Journalism styles are: Straight , Investigative, Commentary. Straight journalism focuses on delivering information without any personal bias or interpretation. Commentary journalism focuses on providing an opinion or analysis, often with a specific angle or bias. Investigative Journalism uncovers hidden information through in-depth research and reporting, often with the goal of revealing systemic issues or wrongdoing.
  Identify how ethnic people are

In [54]:
from tqdm import tqdm

articles = ethnic_df["content"].values.tolist()
responses = []
for article in tqdm(articles[:N]):
    responses.append(get_response(article))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.79s/it]


In [55]:
responses

['{\n"genre": "Crime",\n"journalism_style": "Straight",\n"featured": "Subject",\n"is_ethnic": true,\n"sentiment": "Negative",\n"agenda_setting": "Investigation",\n"framing": "Criminalizing",\n"priming": "Legal",\n"mobilizing": "Community"\n}',
 '{\n  "genre": "Crime",\n  "journalism_style": "Straight",\n  "featured": "Eyewitness",\n  "is_ethnic": false,\n  "sentiment": "Negative",\n  "agenda_setting": "Critical",\n  "framing": "Informative",\n  "priming": "Awareness",\n  "mobilizing": "Action"\n}',
 '{\n  "genre": "Crime",\n  "journalism_style": "Straight",\n  "featured": "Eyewitness",\n  "is_ethnic": true,\n  "sentiment": "Negative",\n  "agenda_setting": "Exposing",\n  "framing": "Conflict",\n  "priming": "Violence",\n  "mobilizing": "Justice"\n}',
 '{\n    "genre": "Crime",\n    "journalism_style": "Straight",\n    "featured": "Subject",\n    "is_ethnic": false,\n    "sentiment": "Negative",\n    "agenda_setting": "Justice",\n    "framing": "Legal proceedings",\n    "priming": "Punis

In [56]:
ethnic_sample_df = ethnic_df.iloc[:N].copy()
ethnic_sample_df["Genre"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["genre"] for response in responses]
ethnic_sample_df["Journalism Style"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["journalism_style"] for response in responses]
ethnic_sample_df["Featured"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["featured"] for response in responses]
ethnic_sample_df["Is_ethnic"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["is_ethnic"] for response in responses]
ethnic_sample_df["Sentiment"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["sentiment"] for response in responses]
ethnic_sample_df["Agenda Setting"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["agenda_setting"] for response in responses]
ethnic_sample_df["Framing"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["framing"] for response in responses]
ethnic_sample_df["Priming"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["priming"] for response in responses]
ethnic_sample_df["Mobilizing"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["mobilizing"] for response in responses]

ethnic_sample_df.head()

,Unnamed: 0,content,Genre,Journalism Style,Featured,Is_ethnic,Sentiment,Agenda Setting,Framing,Priming,Mobilizing
0,0,রমেল হত্যায় মানবাধিকার কমিশনের তদন্ত শুরুরাঙ্গ...,Crime,Straight,Subject,True,Negative,Investigation,Criminalizing,Legal,Community
1,1,"নারায়ণগঞ্জের রূপগঞ্জ ও ফতুল্লা, গাজীপুরের কালি...",Crime,Straight,Eyewitness,False,Negative,Critical,Informative,Awareness,Action
2,2,গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগ...,Crime,Straight,Eyewitness,True,Negative,Exposing,Conflict,Violence,Justice
3,3,চাঞ্চল্যকর পিলখানা হত্যা মামলায় ডেথ রেফারেন্স ...,Crime,Straight,Subject,False,Negative,Justice,Legal proceedings,Punishment,Activism
4,4,অবাধে মাটি কেটে নেয়ার ধ্বংসের দ্বারপ্রান্তে ধো...,Environment,Straight,Subject,True,Negative,Awareness,Conservation,Ecological,Community


In [57]:
ethnic_sample_df.to_csv(f"../data/ethnic_{N}_chatgpt4o_mini_{EXPERIMENT_IDENTIFIER}.csv")

In [58]:
import joblib
joblib.dump(responses, f"../data/ethnic_{N}_chatgpt4o_mini_{EXPERIMENT_IDENTIFIER}.joblib")

['../data/ethnic_5_chatgpt4o_mini_prompt_experiment_1.joblib']

In [45]:
ethnic_sample_df.iloc[2].values.tolist()

[np.int64(2),
 'গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগঞ্জ ইক্ষু খামারে দেয়া কাঁটাতারের বেড়ার পিলার ভাংচুর করেছে দুর্বৃত্তরা।সোমবার রাতে খামার সংলগ্ন জয়পুর পাড়ার দক্ষিণ অংশে দেয়া ২৬টি পিলার ভাংচুর করা হয়।খামারের জমি সাঁওতালদের কাছ থেকে দখলমুক্ত করার পর মিল কর্তৃপক্ষ এই কাঁটাতারের বেড়া দেয়। এরপর সোমবার রাতে সাঁওতাল পল্লী সংলগ্ন খামারের বেড়ার পিলার ভাংচুর করে দুর্বৃত্তরা।রংপুর সুগার মিলের ব্যবস্থাপনা পরিচালক আব্দুল আউয়াল জানান, দুস্কৃতকারীরা রাতের আধারে খামারের পশ্চিম অংশে সাঁওতাল পল্লী জয়পুর পাড়া সংলগ্ন ২৬টি পিলার ভাংচুর করে। এ ঘটনায় ১৩ জনের নাম উল্লেখ করে খামারের উপ-ব্যবস্থাপক মো. হুমায়ন কবির মঙ্গলবার রাতে গোবিন্দগঞ্জ থানায় একটি জিডি করেন।গোবিন্দগঞ্জ থানার ওসি সুব্রত কুমার সরকার বিষয়টি নিশ্চিত করে বলেন, ঘটনার বিষয়টি খতিয়ে দেখা হচ্ছে।গত ৬ নভেম্বর আদিবাসী সাঁওতালদের সঙ্গে আইনশৃংখলা বাহিনী চিনিকলের শ্রমিক-কর্মচারীদের সংঘর্ষের পর ইক্ষু খামার থেকে সাঁওতাল বসতি উচ্ছেদ, লুটপাট, গুলিতে ৩ সাঁওতাল নিহত হওয়ার ঘটনায় সেখানে এখনও অস্থিরতা রয়েছে।',
 'Crime',
 'Investigative',
 'Victim',
 np.Tr

In [78]:
ethnic_sample_df = ethnic_df.sample(n=100, random_state=3)

In [79]:
ethnic_sample_df.to_csv("../data/annotated_100_ethnic_article_against_5_metrics.csv")